In [1]:
import json
import boto3
from botocore.exceptions import ClientError
from datetime import datetime

# Initialize DynamoDB client with explicit region
dynamodb = boto3.resource('dynamodb', region_name='eu-north-1')  # Stockholm region
table = dynamodb.Table('Orders')

def lambda_handler(event, context):
    # Debug: Print incoming event
    print(f"📨 Incoming event: {json.dumps(event)}")

    # Validate event structure
    if 'Records' not in event:
        error_msg = "⚠️ Invalid event format: No 'Records' found"
        print(error_msg)
        return {
            'statusCode': 400,
            'body': json.dumps({'error': error_msg})
        }

    # Process each record
    for record in event['Records']:
        try:
            # Debug: Print raw record
            print(f"📝 Processing record: {record}")

            # Step 1: Parse message body
            try:
                body = json.loads(record['body'])
                print(f"🔍 Parsed body: {body}")
            except json.JSONDecodeError as e:
                error_msg = f"⚠️ JSON decode error: {str(e)}"
                print(error_msg)
                continue

            # Step 2: Handle SNS message unwrapping
            try:
                message = json.loads(body['Message']) if 'Message' in body else body
                print(f"📦 Unwrapped message: {message}")
            except json.JSONDecodeError as e:
                error_msg = f"⚠️ SNS message unwrap error: {str(e)}"
                print(error_msg)
                continue

            # Step 3: Validate required fields
            required_fields = ['orderId', 'userId', 'itemName', 'quantity', 'status', 'timestamp']
            if not all(field in message for field in required_fields):
                missing = [field for field in required_fields if field not in message]
                error_msg = f"⚠️ Missing required fields: {missing}"
                print(error_msg)
                continue

            # Step 4: Validate data types
            try:
                quantity = int(message['quantity'])
                # Validate ISO 8601 timestamp format
                datetime.strptime(message['timestamp'], "%Y-%m-%dT%H:%M:%SZ")
            except ValueError as e:
                error_msg = f"⚠️ Invalid data format: {str(e)}"
                print(error_msg)
                continue

            # Step 5: Write to DynamoDB
            try:
                # First verify table exists
                table.load()
                print(f"✔️ Table {table.name} exists in {table.meta.client.meta.region_name}")

                # Put item
                response = table.put_item(
                    Item={
                        'orderId': message['orderId'],
                        'userId': message['userId'],
                        'itemName': message['itemName'],
                        'quantity': quantity,
                        'status': message['status'],
                        'timestamp': message['timestamp']
                    },
                    ReturnValues='NONE'
                )
                success_msg = f"✅ Successfully processed order {message['orderId']}"
                print(success_msg)
                print(f"📊 DynamoDB response: {response}")

            except ClientError as e:
                error_code = e.response['Error']['Code']
                error_msg = f"❌ DynamoDB error ({error_code}): {e.response['Error']['Message']}"
                print(error_msg)
                if error_code == 'ResourceNotFoundException':
                    print("💡 Solution: Verify table name and region match exactly")
                continue

        except Exception as e:  # This was the line with missing colon
            error_msg = f"⚠️ Unexpected error processing record: {str(e)}"
            print(error_msg)
            continue

    return {
        'statusCode': 200,
        'body': json.dumps({'message': 'Processing complete'})
    }

ModuleNotFoundError: No module named 'boto3'